In [ ]:
#!g1.1
# !git clone https://github.com/nerel-ds/NEREL

In [ ]:
#!g1.1
# !unzip NEREL-v1.0.zip -d NEREL

In [1]:
#!g1.1
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [2]:
#!g1.1
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:04<00:00, 169.33it/s]


In [3]:
#!g1.1
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [5]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
#!g1.1
# pre_dataset = []

# for text_id in range(len(texts)):
#     relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
#     nes = []
#     for i in range(len(entities[text_id])):
#         # O(n^2) eeeeeeeeee
#         for j in range(i+1, len(entities[text_id])):
#             if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
#                 nes.append((entities[text_id][i], entities[text_id][j]))
#     for e in nes:
#         if (e in relationships_nested):
#             pre_dataset.append((*e, relationships_nested[e].tag))
#         elif ((e[1], e[0]) in relationships_nested):
#             pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag))
#         else:
#             pre_dataset.append((*e, 'None'))
# #             pre_dataset.append((e[1], e[0], 'None'))

# Dataset

In [6]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [26]:
#!g1.1
from transformers import BertModel, BertTokenizer

In [8]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [9]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [21]:
#!g1.1
MAX_LENGTH = 150

In [22]:
#!g1.1
class Binary_Tag:
    def __getitem__(self, x):
        if (x == 'None'): return 0
        else: return 1

# tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = Binary_Tag() # string tag --> ids

def tok(text1):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]
    
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*e, relationships_nested[e].tag, text_id))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag, text_id))
                else:
                    pre_dataset.append((*e, 'None', text_id))

#         random.seed(2021)
        data = []
        for e1, e2, tag, text_id in tqdm(pre_dataset):
            
            if (len(e1.text) < len(e2.text)):
                e1,e2 = e2, e1
            
            tag1, tag2 = entity_tags.index(e1.tag), entity_tags.index(e2.tag)
            
            text = texts[text_id][max(e1.start1 - 100,0): e1.end1 + 100]
#             while (text[0] != ' ' and not e1.start1-50 <= 0): text = text[1:]
#             while (text[-1] != ' '): text = text[:-1]
            subtext = ' { ' + e1.text.replace(e2.text, '[ ' + e2.text + ' ]') + ' } '
            text = text.replace(e1.text, subtext)
#             print(text)
            data.append((float(reversed_tags[tag]), tag1, tag2, *tok(text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [23]:
#!g1.1
train_data = MyDataset('NEREL/NEREL-v1.0/train')
train_dl = DataLoader(train_data, shuffle=True, batch_size=32)

dev_data = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl = DataLoader(dev_data, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text






In [20]:
#!g1.1
next(iter(dev_dl))

[tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0.],
        dtype=torch.float64),
 tensor([23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 23, 23, 24, 20, 20, 12, 12, 12,
         12, 24, 24, 16, 16,  6, 23, 23, 23, 23, 21, 21, 21, 23]),
 tensor([23, 17, 23, 17, 24, 24, 24, 17, 24, 17, 17, 23, 27, 28, 28, 22, 23, 12,
         24, 24, 17,  3, 16,  3, 24, 24, 23, 17,  8,  8,  8, 23]),
 tensor([[  101,   284,   222,  ...,     0,     0,     0],
         [  101,   284, 17710,  ...,     0,     0,     0],
         [  101, 17710,   876,  ...,     0,     0,     0],
         ...,
         [  101,   879,  2739,  ...,     0,     0,     0],
         [  101, 13886,  7275,  ...,     0,     0,     0],
         [  101,  5247,  2739,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
   

In [ ]:
#!g1.1
next(iter(dev_dl))[1].size()

# Training

In [ ]:
#!g1.1
# from transformers import BertTokenizer, BertModel
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# inputs = tokenizer.encode_plus("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# pooled_output = outputs[1]

In [27]:
#!g1.1
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            max_length = MAX_LENGTH
            )
        self.dropout = nn.Dropout(0.1)
        self.emb = nn.Embedding(29, 10)
        self.fc = nn.Sequential(
            nn.Linear(768 + 10*2, 100),
            nn.LeakyReLU(True),
            nn.Linear(100, 1)
            )
        
    def forward(self, ent1, ent2, input_ids, token_type_ids, attention_mask):
        ent1 = self.emb(ent1)
        ent2 = self.emb(ent2)
        out = self.bert(input_ids = input_ids, token_type_ids= token_type_ids, attention_mask = attention_mask)[1]
        out = self.dropout(out)
        out = torch.cat((out, ent1, ent2), dim=1)
        out = self.fc(out)
        return out

In [32]:
#!g1.1
model = MyModel()
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [ ]:
#!g1.1
# model(torch.tensor([1]), torch.tensor([1]), **tokenizer.encode_plus("привет", return_tensors="pt"))

In [ ]:
#!g1.1
tokenizer.encode_plus("привет", return_tensors="pt")

In [29]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.round(torch.sigmoid(y_pred)) == y)*1.0/len(y)

def train():
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
        
#         print(outputs.shape, labels.shape)
        outputs = outputs.view(-1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
#         print(outputs, labels)
        epoch_accuracy += calculate_accuracy(outputs, labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
    print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

    
from sklearn.metrics import f1_score
def test():
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

        dev_dl = DataLoader(dev_data, batch_size=32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)
            
            outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
            
            
            outputs = outputs.view(-1)
            loss = criterion(outputs, labels)
            calculate_accuracy(outputs, labels)
            
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.round(torch.sigmoid(outputs))]

            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs, labels).item()
            # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')

In [33]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, patience=3, cooldown = 2, factor = 0.5)

In [34]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 21):
    print('lr=', optimizer.param_groups[0]['lr'])
    print(f'epoch = {epoch}')
    train()
    test()

lr= 5e-05
epoch = 1

train loss= 0.6633 
 train accuracy = 0.6114,

test loss= 0.6271 
 test accuracy = 0.6859, F1 = 0.3639191290824261
lr= 5e-05
epoch = 2

train loss= 0.6261 
 train accuracy = 0.6751,

test loss= 0.5982 
 test accuracy = 0.7072, F1 = 0.48097826086956524
lr= 5e-05
epoch = 3

train loss= 0.6024 
 train accuracy = 0.6815,

test loss= 0.5760 
 test accuracy = 0.7042, F1 = 0.482573726541555
lr= 5e-05
epoch = 4

train loss= 0.5843 
 train accuracy = 0.6838,

test loss= 0.5621 
 test accuracy = 0.7149, F1 = 0.5338345864661653
lr= 5e-05
epoch = 5

train loss= 0.5696 
 train accuracy = 0.6868,

test loss= 0.5464 
 test accuracy = 0.7156, F1 = 0.534504391468005
lr= 5e-05
epoch = 6

train loss= 0.5588 
 train accuracy = 0.6925,

test loss= 0.5389 
 test accuracy = 0.6931, F1 = 0.5753715498938429
lr= 5e-05
epoch = 7

train loss= 0.5508 
 train accuracy = 0.6916,

test loss= 0.5311 
 test accuracy = 0.7351, F1 = 0.6754221388367729
lr= 5e-05
epoch = 8

train loss= 0.5440 
 train a

In [38]:
#!g1.1
def test2():
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

        dev_dl = DataLoader(dev_data, batch_size=32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)
            
            outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
            
            
            outputs = outputs.view(-1)
            loss = criterion(outputs, labels)
            calculate_accuracy(outputs, labels)
            
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.round(torch.sigmoid(outputs))]

            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs, labels).item()
            # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')
    
test2()


test loss= 0.5048 
 test accuracy = 0.7095, F1 = 0.5338253382533825


In [50]:
#!g1.1
torch.save(model.state_dict(), 'models/Bert_binary_nested_only_ent_tags_context')

In [51]:
#!g1.1
for epoch in range(21, 41):
    print('lr=', optimizer.param_groups[0]['lr'])
    print(f'epoch = {epoch}')
    train()
    test()

lr= 5e-05
epoch = 21



train loss= 0.5097 
 train accuracy = 0.7462,



test loss= 0.4982 
 test accuracy = 0.7324, F1 = 0.6812386156648451
lr= 5e-05
epoch = 22



train loss= 0.5077 
 train accuracy = 0.7481,

test loss= 0.4974 
 test accuracy = 0.7507, F1 = 0.6553911205073996
lr= 5e-05
epoch = 23



train loss= 0.5053 
 train accuracy = 0.7523,

test loss= 0.5055 
 test accuracy = 0.7401, F1 = 0.7089041095890412
lr= 5e-05
epoch = 24


...


test loss= 0.4926 
 test accuracy = 0.7317, F1 = 0.689655172413793
lr= 5e-05
epoch = 31



train loss= 0.5002 
 train accuracy = 0.7494,



test loss= 0.4923 
 test accuracy = 0.7309, F1 = 0.6890459363957597
lr= 5e-05
epoch = 32



train loss= 0.5003 
 train accuracy = 0.7503,

test loss= 0.4915 
 test accuracy = 0.7351, F1 = 0.6871609403254972
lr= 5e-05
epoch = 33


KeyboardInterrupt: 

In [52]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5/2, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, patience=3, cooldown = 2, factor = 0.5)

In [53]:
#!g1.1
for epoch in range(33, 41):
    print('lr=', optimizer.param_groups[0]['lr'])
    print(f'epoch = {epoch}')
    train()
    test()

lr= 2.5e-05
epoch = 33



train loss= 0.4988 
 train accuracy = 0.7508,



test loss= 0.4920 
 test accuracy = 0.7614, F1 = 0.6770186335403726
lr= 2.5e-05
epoch = 34



train loss= 0.4998 
 train accuracy = 0.7499,



test loss= 0.4925 
 test accuracy = 0.7385, F1 = 0.7026086956521739
lr= 2.5e-05
epoch = 35


...


train loss= 0.4978 
 train accuracy = 0.7554,

test loss= 0.4916 
 test accuracy = 0.7385, F1 = 0.7026086956521739
lr= 2.5e-05
epoch = 39



train loss= 0.4978 
 train accuracy = 0.7495,



test loss= 0.4911 
 test accuracy = 0.7355, F1 = 0.6916221033868093
lr= 2.5e-05
epoch = 40



train loss= 0.4986 
 train accuracy = 0.7477,



test loss= 0.4910 
 test accuracy = 0.7362, F1 = 0.6922390722569135


In [49]:
#!g1.1
from sklearn.metrics import f1_score

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader(dev_data, batch_size=1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []

    for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)

        outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
            

        outputs = outputs.view(-1)
        loss = criterion(outputs, labels)
        calculate_accuracy(outputs, labels)
#         print(torch.round(torch.sigmoid(outputs)).detach().cpu().numpy(), labels)
#         print(input_ids[0].detach().cpu().numpy())
        res = torch.round(torch.sigmoid(outputs)).detach().cpu().item()
        if (res != labels.detach().cpu().numpy()):
            print('expected:', labels.detach().cpu().item())
            print('found:', res, '(', torch.sigmoid(outputs).detach().cpu().numpy() ,')')
            print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))

expected: 1.0
found: 0.0 ( [0.44154993] )
[CLS] зидентом Аргентины Маурисио Макри и его супругой Хулианой Авада 20 мая 2017 года Кронпринц Нарухито { Верхняя палата [ парламента Японии ] } 9 июня 2017 года приняла закон, позволяющий императору Акихито сложить полномочия и передать власть [SEP]                                                                                                    
expected: 0.0
found: 1.0 ( [0.64927036] )
[CLS] зидентом Аргентины Маурисио Макри и его супругой Хулианой Авада 20 мая 2017 года Кронпринц Нарухито { Верхняя палата парламента [ Японии ] } 9 июня 2017 года приняла закон, позволяющий императору Акихито сложить полномочия и передать власть [SEP]                                                                                                    
expected: 1.0
found: 0.0 ( [0.4986793] )
[CLS] Сейчас пара живёт в Барселоне. С тех пор как Месси переехал в Испанию, он завоевал рекордные пять { Золотых мячей [ ФИФА ] }. В мае 2017 года Верховный суд Испании

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [ ]:
#!g1.1
